In [189]:
# 05/08/2025 
# Indices updated, not tested

using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 6
J = 3 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor
α = ones(J) * (1/J) # final good expenditure share

τ = ones(N, N, J) # Iceberg trade costs 
Lt = ones(N, J, periods) # Size of labor force in each country at time 


Ldot = ones(N, J, periods)

At0 = ones(N, J)#initial productivities

wt = ones(N, J, periods)
wt0 = ones(N, J)

wdot = ones(N, J, periods)
tradesharest0 = ones(N, N, J) * (1 / N)
Adot = ones(N, J, periods)



kdot = ones(N, N, J, periods)


pdotArray = ones(N, J, periods)
d1wdot = ones(N, J)

μtminus1 = zeros(N, N, J, J) #value of μ at time t = -1
πt0 = ones(N, N, J) #initial trade shares
μt = ones(N, N, J, J, periods)*(1/(N*J))

# a guess for path of udot:
udotPathGuess = 1.1*ones(N, J, periods+1)
udotPathUpdate = zeros(N, J, periods+1)
errormax = 1.0 #the maximum log difference between guesses and updates for appendix D algorithm


function pdot(n, j, d1wdot, kdot, Adot, time, tradesharest0) #pdot(nj) from equation (12)
    (sum(tradesharest0[n, i, j] * (d1wdot[i, j] * kdot[n, i, j, time])^-θ[j] * Adot[i, j, time]^θ[j] for i in 1:N))^(-1 / θ[j])
end 

function tradeSharest0(n, i, j, wt0, At0, τ) #trade shares to nj from ij, equation (7)
    (wt0[i, j] * τ[n, i, j])^-θ[j] * At0[i, j] ^θ[j] / (sum((wt0[m, j] *τ[n, m, j])^-θ[j] * At0[m, j]^θ[j] for m in 1:N)) 
end

function tradeSharest1(n, i, j, d1wdot, Ldot, Adot, kdot, time, tradesharest0) #trade shares to nj from ij, equation (13)
    tradesharest0[n, i, j] * ((d1wdot[i, j] * kdot[n, i, j, time]) / pdot(n, j, d1wdot, kdot, Adot, time, tradesharest0))^-θ[j] * Adot[i, j, time]^θ[j]
end

function incomet0(n, j, wt0, Lt) # total income of country n, sector j in time t=0 given wage and labor  
    wt0[n, j] * Lt[n, j, 1] 
end

function incomet1(n, j, d1wdot, time, Lt, wt) # total income of country n, sector j in time t+1
    wt[n, j, time]*(d1wdot[n, j])*Lt[n, j, time]*(Ldot[n, j, time])
end

function Xt0(n, j, α, wt0, Lt) # expenditure on sector good j in region n, from equation (8)
    α[j] * sum(wt0[n, k] * Lt[n, k, 1] for k in 1:J) 
end

function Xt1(n, j, α, d1wdot, Lt, wt, Ldot, time) # expenditure on sector good j in region n, from equation (14)
    α[j] * sum(d1wdot[n, k] * Ldot[n, k, time] * wt[n, k, time] * Lt[n, k, time] for k in 1:J) #from equation (14) 
end


while errormax > .00001 
    for time in 1:periods-1
        # Update μt step 2 of appendix D
        for n in 1:N               # destination country
            for i in 1:N           # source country
                for j in 1:J       # destination sector
                    for k in 1:J   # source sector
                        num = μt[n, i, j, k, time] *
                            (udotPathGuess[i, k, time+1])^(β/v)

                        # denominator: sum over all (m, h)
                        denom = sum(μt[n, m, j, h, time] * (udotPathGuess[m, h, time+1])^(β/v) for m in 1:N, h in 1:J)
                        μt[n, i, j, k, time+1] = num / denom
                    end
                end
            end
        end

        # Update Lt step 3 of appendix D
        for n in 1:N
            for j in 1:J
                Lt[n, j, time+1] = sum(μt[i, n, k, j, time] * Lt[i, k, time] for i in 1:N, k in 1:J)
            end
        end
        # Update Ldots from Lts
        Ldot[:, :, time] .= Lt[:, :, time+1] ./ Lt[:, :, time]
    end

    function g!(G, wt0)
        for n in 1:N
            for j in 1:J
                G[n, j] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N) #market clearing for wt0
            end
        end
    end

    initial = [1.0  2.11  1.2;
           1.15 5.22 1.21]    # size: 2×3 if N=2, J=3
    results = nlsolve(g!, initial) #solve for wages

    # extract the wage solution as an N×J matrix
    wt0 = results.zero
    
    wt[:,:, 1] .= wt0[:,:]

    for n in 1:N
        for i in 1:N
            for j in 1:J
                tradesharest0[n, i , j] = tradeSharest0(n, i, j, wt0, At0, τ)
            end
        end
    end

    println(wt0)  #end solving for wt0

    for time in 1:periods-1
        function f_with_norm!(F, x)
            # Unpack the first N*J entries into a matrix:
            d1 = reshape(x[1:N*J], N, J)
            # Fill your first N*J equations (market clearing)
            idx = 1
            for n in 1:N, j in 1:J
                F[idx] = incomet1(n, j, d1, time, Lt, wt) -
                         sum(
                           tradeSharest1(n, i, j, d1, Ldot, Adot, kdot, time, tradesharest0)
                           * Xt1(i, j, α, d1, Lt, wt, Ldot, time)
                           for i in 1:N
                         )
                idx += 1
            end
            # The last equation pins d1[1,1] to 1.0
            F[N*J + 1] = d1[1,1] - 1.0
        end
        
        # initial guess: ones for 6 growth‐rates, plus one dummy
        x0 = 1.1*ones(N*J + 1)
        
        # solve
        res = nlsolve(f_with_norm!, x0)
        
        # extract your d1wdot matrix:
        d1wdot = reshape(res.zero[1:N*J], N, J)

        println("checkpoint")

        wdot[:,:,time] .= d1wdot[:,:] 

        for n in 1:N, j in 1:J
            println(wdot[n, j, time])
        end

        println("checkpoint")
        for n in 1:N, j in 1:J
            pdotArray[n,j,time] = pdot(n, j, d1wdot, kdot, Adot,time, tradesharest0)
        end
    
        tradesharest0[:,:,:] .= [tradeSharest1(n,i,j,d1wdot,Ldot,Adot,kdot,time,tradesharest0) for n in 1:N, i in 1:N, j in 1:J]

        wt[:, :, time + 1] .= wdot[:, :, time] .* wt[:, :, time]

    end
    println("udotPathGuess = ", udotPathGuess)
    for time in 1:periods-1
        for n in 1:N, j in 1:J
            udotPathUpdate[n,j,time] = wdot[n,j,time]*(sum(μt[n,i,j,k,time]*(udotPathGuess[i,k,time+1])^(β/v) for i in 1:N, k in 1:J))^(v) ##needs verification but is equation 17 as specified by step 5
        end## Note: trying to figure out how to make sure the time+1 does not end the program with infs or NaN in the error
    end
    println("udotPathUpdate = ", udotPathUpdate)
    
    udotPathUpdate[:,:,periods] .= udotPathGuess[:,:,periods] # makes it so that udotPathUpdate in time 6 never goes to 0

    # Take the log difference of the guess and updated udots to get the error
    logudotPathGuess = log.(udotPathGuess) 

    logudotPathUpdate = log.(udotPathUpdate)
    logdifference = abs.(logudotPathGuess[:,:,1:5] - logudotPathUpdate[:,:,1:5])
    errormax = maximum(logdifference[:,:,1:5])

    udotPathGuess = udotPathUpdate
    println("WHILE LOOP")
    display(errormax)
end


[2.1246560066856546e-11 2.6850521805954486e-11 -1.8577361871052744e-11; 5.0934145789938157e-11 -6.111111616746712e-11 2.1230572855301943e-11]
checkpoint
1.0000000000003553
0.44104979147815804
-1.0022748615457329
0.28666423667546914
-0.23865650414601247
1.0429966998375304
checkpoint
checkpoint
1.0000000000003553
1.7881756103767472
1.1381499891188034
1.4551225682441875
1.4519543000494168
0.9570179282150456
checkpoint
checkpoint
1.0000000000003553
1.0033166961649167
1.0025780230398555
1.000014826303705
1.0033261224489374
1.0025910814373176
checkpoint
checkpoint
1.0000000000003553
1.0000006940348325
1.0000006939412063
1.0000006941703774
1.0000006940438764
1.0000006939493629
checkpoint
checkpoint
1.0000000000003553
0.9999999783979958
0.999999978405075
0.9999999783955142
0.9999999783968542
0.999999978405164
checkpoint
udotPathGuess = [1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.

DomainError: DomainError with -0.26227206342316417:
log was called with a negative real argument but will only return a complex result if called with a complex argument. Try log(Complex(x)).

In [191]:
using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 6
J = 3 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor
α = ones(J) * (1/J) # final good expenditure share

τ = ones(N, N, J) # Iceberg trade costs 
Lt = ones(N, J, periods) # Size of labor force in each country at time 
Lt[1,1,1] = 1.1

Ldot = ones(N, J, periods)

At0 = ones(N, J)#initial productivities

wt = ones(N, J, periods)
wt0 = ones(N, J)

wdot = ones(N, J, periods)
tradesharest0 = ones(N, N, J) * (1 / N)
Adot = ones(N, J, periods)
Adot[1,1,:] .= 1.5

kdot = ones(N, N, J, periods)
pdotArray = ones(N, J, periods)
d1wdot = ones(N, J)

μtminus1 = zeros(N, N, J, J) #value of μ at time t = -1
πt0 = ones(N, N, J) #initial trade shares
μt = ones(N, N, J, J, periods)*(1/(N*J))

# a guess for path of udot:
udotPathGuess = 1.1*ones(N, J, periods+1)
udotPathUpdate = zeros(N, J, periods+1)
errormax = 1.0 #the maximum log difference between guesses and updates for appendix D algorithm


function pdot(n, j, d1wdot, kdot, Adot, time, tradesharest0) #pdot(nj) from equation (12)
    (sum(tradesharest0[n, i, j] * (d1wdot[i, j] * kdot[n, i, j, time])^-θ[j] * Adot[i, j, time]^θ[j] for i in 1:N))^(-1 / θ[j])
end 

function tradeSharest0(n, i, j, wt0, At0, τ) #trade shares to nj from ij, equation (7)
    (wt0[i, j] * τ[n, i, j])^(-θ[j]) * At0[i, j] ^θ[j] / (sum((wt0[m, j] *τ[n, m, j])^(-θ[j]) * At0[m, j]^θ[j] for m in 1:N)) 
end

function tradeSharest1(n, i, j, d1wdot, Ldot, Adot, kdot, time, tradesharest0) #trade shares to nj from ij, equation (13)
    tradesharest0[n, i, j] * ((d1wdot[i, j] * kdot[n, i, j, time]) / pdot(n, j, d1wdot, kdot, Adot, time, tradesharest0))^-θ[j] * Adot[i, j, time]^θ[j]
end

function incomet0(n, j, wt0, Lt) # total income of country n, sector j in time t=0 given wage and labor  
    wt0[n, j] * Lt[n, j, 1] 
end

function incomet1(n, j, d1wdot, time, Lt, wt) # total income of country n, sector j in time t+1
    wt[n, j, time]*(d1wdot[n, j])*Lt[n, j, time]*(Ldot[n, j, time])
end

function Xt0(n, j, α, wt0, Lt) # expenditure on sector good j in region n, from equation (8)
    α[j] * sum(wt0[n, k] * Lt[n, k, 1] for k in 1:J) 
end

function Xt1(n, j, α, d1wdot, Lt, wt, Ldot, time) # expenditure on sector good j in region n, from equation (14)
    α[j] * sum(d1wdot[n, k] * Ldot[n, k, time] * wt[n, k, time] * Lt[n, k, time] for k in 1:J) #from equation (14) 
end


while errormax > .00001 
    for time in 1:periods-1
        # Update μt step 2 of appendix D
        for n in 1:N               # destination country
            for i in 1:N           # source country
                for j in 1:J       # destination sector
                    for k in 1:J   # source sector
                        num = μt[n, i, j, k, time] *
                            (udotPathGuess[i, k, time+1])^(β/v)

                        # denominator: sum over all (m, h)
                        denom = sum(μt[n, m, j, h, time] * (udotPathGuess[m, h, time+1])^(β/v) for m in 1:N, h in 1:J)
                        μt[n, i, j, k, time+1] = num / denom
                    end
                end
            end
        end

        # Update Lt step 3 of appendix D
        for n in 1:N
            for j in 1:J
                Lt[n, j, time+1] = sum(μt[i, n, k, j, time] * Lt[i, k, time] for i in 1:N, k in 1:J)
            end
        end
        # Update Ldots from Lts
        Ldot[:, :, time] .= Lt[:, :, time+1] ./ Lt[:, :, time]
    end

    function g!(G, wt0)
        # Unpack the first N*J entries into a matrix:
        wt0 = reshape(wt0[1:N*J], N, J)
        # Fill your first N*J equations (market clearing)
        idx = 1
        for n in 1:N, j in 1:J
            G[idx] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N)
            idx += 1
        end
        # The last equation pins d1[1,1] to 1.0
        G[N*J + 1] = wt0[1,1] - 1.0
    end
    
    initial = [1.0, 2.11, 1.2, 1.15, 5.22, 1.21, 1.01]    # size: 2×3 if N=2, J=3
    results = nlsolve(g!, initial) #solve for wages
    
    # extract the wage solution as an N×J matrix
    wt0 = reshape(results.zero[1:N*J], N, J)
    
    wt[:,:, 1] .= wt0[:,:]

    for n in 1:N
        for i in 1:N
            for j in 1:J
                tradesharest0[n, i , j] = tradeSharest0(n, i, j, wt0, At0, τ)
            end
        end
    end

    println(wt0)  #end solving for wt0

    for time in 1:periods-1
        function f!(F, d1wdot)
            for n in 1:N
                for j in 1:J
                    F[n,j] = (incomet1(n, j, d1wdot, time, Lt, wt) - sum(tradeSharest1(n, i, j, d1wdot, Ldot, Adot, kdot, time, tradesharest0)
                    * Xt1(i, j, α, d1wdot, Lt, wt, Ldot, time) for i in 1:N)) # equation (15)
                end
            end
        end

        println("checkpoint")

        initial = [1.0  2.11  1.2;
           1.15 5.22 1.21] 
        res_f = nlsolve(f!, initial)
        #println("Random guesses:", initial_rand)

        wdot[:, :, time] .= res_f.zero[:,:] # updates wdot with solution to the system of equations
        d1wdot[:,:] .= wdot[:,:,time] 

        for n in 1:N, j in 1:J
            println(wdot[n, j, time])
        end

        println("checkpoint")
        ################### not edited past this point
        for n in 1:N, j in 1:J
            pdotArray[n,j,time] = pdot(n, j, d1wdot, kdot, Adot,time, tradesharest0)
        end
    
        tradesharest0[:,:,:] .= [tradeSharest1(n,i,j,d1wdot,Ldot,Adot,kdot,time,tradesharest0) for n in 1:N, i in 1:N, j in 1:J]

        wt[:, :, time + 1] .= wdot[:, :, time] .* wt[:, :, time]

    end
    println("udotPathGuess = ", udotPathGuess)
    for time in 1:periods-1
        for n in 1:N, j in 1:J
            udotPathUpdate[n,j,time] = wdot[n,j,time]*(sum(μt[n,i,j,k,time]*(udotPathGuess[i,k,time+1])^(β/v) for i in 1:N, k in 1:J))^(v) ##needs verification but is equation 17 as specified by step 5
        end## Note: trying to figure out how to make sure the time+1 does not end the program with infs or NaN in the error
    end
    println("udotPathUpdate = ", udotPathUpdate)
    
    udotPathUpdate[:,:,periods] .= udotPathGuess[:,:,periods] # makes it so that udotPathUpdate in time 6 never goes to 0

    # Take the log difference of the guess and updated udots to get the error
    logudotPathGuess = log.(udotPathGuess) 

    logudotPathUpdate = log.(udotPathUpdate)
    logdifference = abs.(logudotPathGuess[:,:,1:5] - logudotPathUpdate[:,:,1:5])
    errormax = maximum(logdifference[:,:,1:5])

    udotPathGuess = udotPathUpdate
    println("WHILE LOOP")
    display(errormax)
end

[1.0 1.1000000000020245 1.1000000000023964; 1.1000000000072754 1.1000000000092995 1.100000000009672]
checkpoint
3.5573299861368923e-10
3.0543922946435487e-10
2.2504709207282758e-10
3.298559203557261e-10
3.311786400672645e-10
2.6035484879116666e-10
checkpoint
checkpoint
1.0
2.11
1.2
1.15
5.22
1.21
checkpoint
checkpoint
1.0
2.11
1.2
1.15
5.22
1.21
checkpoint
checkpoint
3.0795760386226334e-9
6.901372806566997e-10
2.0614476792246705e-9
2.28799934554047e-9
8.946798857323301e-11
2.1832506913455063e-9
checkpoint
checkpoint
1.0
2.11
1.2
1.15
5.22
1.21
checkpoint
udotPathGuess = [1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1]
udotPathUpdate = [3.9093352141385896e-10 3.356630788197027e-10 2.4731597161589233e-10; 3.6249585224425423e-10 3.639494578324071e-10 2.861175045659282e-10;;; 1.0989520874851308 2.318788904593626 1.318742504982157; 1.2637949006079

23.138093325997616

DomainError: DomainError with -2.814332580964548e-18:
log was called with a negative real argument but will only return a complex result if called with a complex argument. Try log(Complex(x)).

In [136]:
# 05/08/2025 
# Indices updated, not tested

using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 6
J = 3 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor
α = ones(J) * (1/J) # final good expenditure share

τ = ones(N, N, J) # Iceberg trade costs 
Lt = ones(N, J, periods) # Size of labor force in each country at time 
Lt[1,1,1] = 1.1

Ldot = ones(N, J, periods)

At0 = ones(N, J)#initial productivities

wt = ones(N, J, periods)
wt0 = ones(N, J)

wdot = ones(N, J, periods)
tradesharest0 = ones(N, N, J) * (1 / N)
Adot = ones(N, J, periods)
Adot[1,1,:] .= 1.5

kdot = ones(N, N, J, periods)
pdotArray = ones(N, J, periods)
d1wdot = ones(N, J)

μtminus1 = zeros(N, N, J, J) #value of μ at time t = -1
πt0 = ones(N, N, J) #initial trade shares
μt = ones(N, N, J, J, periods)*(1/(N*J))

# a guess for path of udot:
udotPathGuess = 1.1*ones(N, J, periods+1)
udotPathUpdate = zeros(N, J, periods+1)
errormax = 1.0 #the maximum log difference between guesses and updates for appendix D algorithm


function pdot(n, j, d1wdot, kdot, Adot, time, tradesharest0) #pdot(nj) from equation (12)
    (sum(tradesharest0[n, i, j] * (d1wdot[i, j] * kdot[n, i, j, time])^-θ[j] * Adot[i, j, time]^θ[j] for i in 1:N))^(-1 / θ[j])
end 

function tradeSharest0(n, i, j, wt0, At0, τ) #trade shares to nj from ij, equation (7)
    (wt0[i, j] * τ[n, i, j])^(-θ[j]) * At0[i, j] ^θ[j] / (sum((wt0[m, j] *τ[n, m, j])^(-θ[j]) * At0[m, j]^θ[j] for m in 1:N)) 
end

function tradeSharest1(n, i, j, d1wdot, Ldot, Adot, kdot, time, tradesharest0) #trade shares to nj from ij, equation (13)
    tradesharest0[n, i, j] * ((d1wdot[i, j] * kdot[n, i, j, time]) / pdot(n, j, d1wdot, kdot, Adot, time, tradesharest0))^-θ[j] * Adot[i, j, time]^θ[j]
end

function incomet0(n, j, wt0, Lt) # total income of country n, sector j in time t=0 given wage and labor  
    wt0[n, j] * Lt[n, j, 1] 
end

function incomet1(n, j, d1wdot, time, Lt, wt) # total income of country n, sector j in time t+1
    wt[n, j, time]*(d1wdot[n, j])*Lt[n, j, time]*(Ldot[n, j, time])
end

function Xt0(n, j, α, wt0, Lt) # expenditure on sector good j in region n, from equation (8)
    α[j] * sum(wt0[n, k] * Lt[n, k, 1] for k in 1:J) 
end

function Xt1(n, j, α, d1wdot, Lt, wt, Ldot, time) # expenditure on sector good j in region n, from equation (14)
    α[j] * sum(d1wdot[n, k] * Ldot[n, k, time] * wt[n, k, time] * Lt[n, k, time] for k in 1:J) #from equation (14) 
end


while errormax > .00001 
    for time in 1:periods-1
        # Update μt step 2 of appendix D
        for n in 1:N               # destination country
            for i in 1:N           # source country
                for j in 1:J       # destination sector
                    for k in 1:J   # source sector
                        num = μt[n, i, j, k, time] *
                            (udotPathGuess[i, k, time+1])^(β/v)

                        # denominator: sum over all (m, h)
                        denom = sum(μt[n, m, j, h, time] * (udotPathGuess[m, h, time+1])^(β/v) for m in 1:N, h in 1:J)
                        μt[n, i, j, k, time+1] = num / denom
                    end
                end
            end
        end

        # Update Lt step 3 of appendix D
        for n in 1:N
            for j in 1:J
                Lt[n, j, time+1] = sum(μt[i, n, k, j, time] * Lt[i, k, time] for i in 1:N, k in 1:J)
            end
        end
        # Update Ldots from Lts
        Ldot[:, :, time] .= Lt[:, :, time+1] ./ Lt[:, :, time]
    end

    function g!(G, wt0)
        # Unpack the first N*J entries into a matrix:
        wt0 = reshape(wt0[1:N*J], N, J)
        # Fill your first N*J equations (market clearing)
        idx = 1
        for n in 1:N, j in 1:J
            G[idx] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N)
            idx += 1
        end
        # The last equation pins d1[1,1] to 1.0
        G[N*J + 1] = wt0[1,1] - 1.0
    end
    
    initial = [1.0, 2.11, 1.2, 1.15, 5.22, 1.21, 1.01]    # size: 2×3 if N=2, J=3
    results = nlsolve(g!, initial) #solve for wages
    
    # extract the wage solution as an N×J matrix
    wt0 = reshape(results.zero[1:N*J], N, J)
    
    wt[:,:, 1] .= wt0[:,:]

    for n in 1:N
        for i in 1:N
            for j in 1:J
                tradesharest0[n, i , j] = tradeSharest0(n, i, j, wt0, At0, τ)
            end
        end
    end

    println(wt0)  #end solving for wt0

    for time in 1:periods-1
        function f!(F, d1wdot)
            for n in 1:N
                for j in 1:J
                    F[n,j] = (incomet1(n, j, d1wdot, time, Lt, wt) - sum(tradeSharest1(n, i, j, d1wdot, Ldot, Adot, kdot, time, tradesharest0)
                    * Xt1(i, j, α, d1wdot, Lt, wt, Ldot, time) for i in 1:N)) # equation (15)
                end
            end
        end

        println("checkpoint")

        initial = [1.0  2.11  1.2;
           1.15 5.22 1.21] 
        res_f = nlsolve(f!, initial)
        #println("Random guesses:", initial_rand)

        wdot[:, :, time] .= res_f.zero[:,:] # updates wdot with solution to the system of equations
        d1wdot[:,:] .= wdot[:,:,time] 

        for n in 1:N, j in 1:J
            println(wdot[n, j, time])
        end

        println("checkpoint")
        ################### not edited past this point
        for n in 1:N, j in 1:J
            pdotArray[n,j,time] = pdot(n, j, d1wdot, kdot, Adot,time, tradesharest0)
        end
    
        tradesharest0[:,:,:] .= [tradeSharest1(n,i,j,d1wdot,Ldot,Adot,kdot,time,tradesharest0) for n in 1:N, i in 1:N, j in 1:J]

        wt[:, :, time + 1] .= wdot[:, :, time] .* wt[:, :, time]

    end
    println("udotPathGuess = ", udotPathGuess)
    for time in 1:periods-1
        for n in 1:N, j in 1:J
            udotPathUpdate[n,j,time] = wdot[n,j,time]*(sum(μt[n,i,j,k,time]*(udotPathGuess[i,k,time+1])^(β/v) for i in 1:N, k in 1:J))^(v) ##needs verification but is equation 17 as specified by step 5
        end## Note: trying to figure out how to make sure the time+1 does not end the program with infs or NaN in the error
    end
    println("udotPathUpdate = ", udotPathUpdate)
    
    udotPathUpdate[:,:,periods] .= udotPathGuess[:,:,periods] # makes it so that udotPathUpdate in time 6 never goes to 0

    # Take the log difference of the guess and updated udots to get the error
    logudotPathGuess = log.(udotPathGuess) 

    logudotPathUpdate = log.(udotPathUpdate)
    logdifference = abs.(logudotPathGuess[:,:,1:5] - logudotPathUpdate[:,:,1:5])
    errormax = maximum(logdifference[:,:,1:5])

    udotPathGuess = udotPathUpdate
    println("WHILE LOOP")
    display(errormax)
end


[1.0 1.1000000000020245 1.1000000000023964; 1.1000000000072754 1.1000000000092995 1.100000000009672]
checkpoint
3.5573299861368923e-10
3.0543922946435487e-10
2.2504709207282758e-10
3.298559203557261e-10
3.311786400672645e-10
2.6035484879116666e-10
checkpoint
checkpoint
1.0
2.11
1.2
1.15
5.22
1.21
checkpoint
checkpoint
1.0
2.11
1.2
1.15
5.22
1.21
checkpoint
checkpoint
3.0795760386226334e-9
6.901372806566997e-10
2.0614476792246705e-9
2.28799934554047e-9
8.946798857323301e-11
2.1832506913455063e-9
checkpoint
checkpoint
1.0
2.11
1.2
1.15
5.22
1.21
checkpoint
udotPathGuess = [1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1;;; 1.1 1.1 1.1; 1.1 1.1 1.1]
udotPathUpdate = [3.9093352141385896e-10 3.356630788197027e-10 2.4731597161589233e-10; 3.6249585224425423e-10 3.639494578324071e-10 2.861175045659282e-10;;; 1.0989520874851308 2.318788904593626 1.318742504982157; 1.2637949006079

23.138093325997616

DomainError: DomainError with -2.814332580964548e-18:
log was called with a negative real argument but will only return a complex result if called with a complex argument. Try log(Complex(x)).

In [190]:
# Toy Model

using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 6
J = 3 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor
α = ones(J) * (1/J) # final good expenditure share

Lt = ones(N, J, periods) # Size of labor force in each country at time 

# assume N, J are defined
τ = fill(1.0, N, N, J)    # 1.1 for all n≠i and goods j

for n in 1:N
    τ[n, n, :] .= 1.0     # set τ[n,n,j]=1.0 for every good j
end

Lt[2,1,1] = 1.2


#At0 = ones(N, J)#initial productivities
At0 = [ 1.0  1.0 1.0
        1.0  1.0 1.0 ]

wt = ones(N, J, periods)
wt0 = ones(N, J)

tradesharest0 = ones(N, N, J) * (1 / N)


μtminus1 = zeros(N, N, J, J) #value of μ at time t = -1
πt0 = ones(N, N, J) #initial trade shares
μt = ones(N, N, J, J, periods)*(1/(N*J))



function tradeSharest0(n, i, j, wt0, At0, τ) #trade shares to nj from ij, equation (7)
    (wt0[i, j] * τ[n, i, j])^(-θ[j]) * At0[i, j] ^θ[j] / (sum((wt0[m, j] *τ[n, m, j])^(-θ[j]) * At0[m, j]^θ[j] for m in 1:N)) 
end


function incomet0(n, j, wt0, Lt) # total income of country n, sector j in time t=0 given wage and labor  
    wt0[n, j] * Lt[n, j, 1] 
end



function Xt0(n, j, α, wt0, Lt) # expenditure on sector good j in region n, from equation (8)
    α[j] * sum(wt0[n, k] * Lt[n, k, 1] for k in 1:J) 
end

#=
function g!(G, wt0)
    for n in 1:N
        for j in 1:J
            G[n, j] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N) #market clearing for wt0
        end
    end
end
=#

function g!(G, wt0)
    # Unpack the first N*J entries into a matrix:
    wt0 = reshape(wt0[1:N*J], N, J)
    # Fill your first N*J equations (market clearing)
    idx = 1
    for n in 1:N, j in 1:J
        G[idx] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N)
        idx += 1
    end
    # The last equation pins d1[1,1] to 1.0
    G[N*J + 1] = wt0[1,1] - 1.0
end

initial = [1.0, 2.11, 1.2, 1.15, 5.22, 1.21, 1.01]    # size: 2×3 if N=2, J=3
results = nlsolve(g!, initial) #solve for wages

# extract the wage solution as an N×J matrix
wt0 = reshape(results.zero[1:N*J], N, J)

wt[:,:, 1] .= wt0[:,:]

for n in 1:N
    for i in 1:N
        for j in 1:J
            tradesharest0[n, i , j] = tradeSharest0(n, i, j, wt0, At0, τ)
        end
    end
end

println(wt0)  #end solving for wt0

display(tradesharest0)



2×2×3 Array{Float64, 3}:
[:, :, 1] =
 0.325351  0.674649
 0.325351  0.674649

[:, :, 2] =
 0.5  0.5
 0.5  0.5

[:, :, 3] =
 0.5  0.5
 0.5  0.5

[1.0 1.0000000000418683 1.0000000000390432; 0.8333333332859205 0.999999999984976 0.9999999999821461]


In [187]:
# Toy Model: Back to Basics

using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 3 # Number of countries
periods = 1
J = 1 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor
α = ones(J) * (1/J) # final good expenditure share

Lt = ones(N, J, periods) # Size of labor force in each country at time 

# assume N, J are defined
τ = fill(1.0, N, N, J)    # 1.1 for all n≠i and goods j

for n in 1:N
    τ[n, n, :] .= 1.0     # set τ[n,n,j]=1.0 for every good j
end



#At0 = ones(N, J)#initial productivities
At0 = [ 1.0; 1.1; 1.0]


wt = ones(N, J, periods)
wt0 = ones(N, J)

tradesharest0 = ones(N, N, J) * (1 / N)


μtminus1 = zeros(N, N, J, J) #value of μ at time t = -1
πt0 = ones(N, N, J) #initial trade shares
μt = ones(N, N, J, J, periods)*(1/(N*J))



function tradeSharest0(n, i, j, wt0, At0, τ) #trade shares to nj from ij, equation (7)
    (wt0[i, j] * τ[n, i, j])^(-θ[j]) * At0[i, j] ^θ[j] / (sum((wt0[m, j] *τ[n, m, j])^(-θ[j]) * At0[m, j]^θ[j] for m in 1:N)) 
end


function incomet0(n, j, wt0, Lt) # total income of country n, sector j in time t=0 given wage and labor  
    wt0[n, j] * Lt[n, j, 1] 
end


function Xt0(n, j, α, wt0, Lt) # expenditure on sector good j in region n, from equation (8)
    α[j] * sum(wt0[n, k] * Lt[n, k, 1] for k in 1:J) 
end

#=
function g!(G, wt0)
    for n in 1:N
        for j in 1:J
            G[n, j] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N) #market clearing for wt0
        end
    end
end
=#

function g!(G, wt0)
    # Unpack the first N*J entries into a matrix:
    wt0 = reshape(wt0[1:N*J], N, J)
    # Fill your first N*J equations (market clearing)
    idx = 1
    for n in 1:N, j in 1:J
        G[idx] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N)
        idx += 1
    end
    # The last equation pins d1[1,1] to 1.0
    G[N*J + 1] = wt0[1,1] - 1.0
end

initial = [1.0, 2.11, 1.2, 1.15, 5.22, 1.21, 1.01]    # size: 2×3 if N=2, J=3
results = nlsolve(g!, initial) #solve for wages

# extract the wage solution as an N×J matrix
wt0 = reshape(results.zero[1:N*J], N, J)

wt[:,:, 1] .= wt0[:,:]

for n in 1:N
    for i in 1:N
        for j in 1:J
            tradesharest0[n, i , j] = tradeSharest0(n, i, j, wt0, At0, τ)
        end
    end
end

println(wt0)  #end solving for wt0

display(tradesharest0)

balance = 

3×3×1 Array{Float64, 3}:
[:, :, 1] =
 0.288675  0.422649  0.288675
 0.288675  0.422649  0.288675
 0.288675  0.422649  0.288675

[1.0; 0.9999999998310156; 1.00000000000001;;]


In [182]:
using Random, Distributions, Optim, NLsolve, SpecialFunctions

N = 3 # Number of countries
θ = 4 # Frechet shape parameter (governs comparative advantage)
T = ones(N, 1) # Frechet scale parameter (governs absolute advantage)
σ = 2 # Substitution elasticity between goods
τ = ones(N, N) # Iceberg trade costs
L = ones(N, 1) # Size of labor force in each country
T[1] = 5.0

w = ones(Float64, N) #creating an array of N countries' wages initialized with one
w[1] = 1 #assigning country one's wage as 1
#print(w[1])
#print(" ")
#print(w[2])



#the real step 2:
function ϕ(i) #price parameter for country i
    h = 0 #helps in the calculation below
    for z = 1:N
        h = h + T[z] * (w[z] * τ[i, z])^-θ #equation (7) from Eaton-Kortum
    end
    return h
end


#the not real step 2:
function tradeShares(n, i) #computes fraction of goods country n buys from country i
    (T[i] * (w[i] * τ[n, i])^-θ) / ϕ(n) #equation (8) from Eaton-Kortum
end

function printTradeShares(n, i) #prints the trade shares function
    print("fraction of goods country ")
    print(n)
    print(" buys from country ")
    print(i)
    println(":")
    println(tradeShares(n, i))
    return nothing
end


#what we were supposed to do:
function Y(i, wage) #calculates total income in country i
    wage * L[i]
end


#NOT what we were supposed to do:
biTradeBalance = ones(Float64, N, N) #an array storing the bilateral trade balance between countries n and i

function Y(i) #calculates total income in country i
    w[i] * L[i]
end
function exports(n, i) #calculates the exports country n sells to country i
    tradeShares(i, n) * Y(i)
end
function imports(n, i) #calculates the imports country n buys from country i
    tradeShares(n, i) * Y(n)
end

sol = zeros(Float64, N) #the output of the optimization
welfare = zeros(Float64, N) #an array to store equilibrium welfare
perCapitaWelfare = zeros(Float64, N) #an array to store equilibrium welfare
gam = (gamma((θ + 1 - σ) / θ))^(1 / (1 - σ)) #a parameter in the price index function
function priceIndex(i) #the price level in country i
    gam * (ϕ(i))^(-1 / θ)
end
function wageWelfare(i) #calculating equilibrium wage and welfare of country i, storing them in their respective arrays
    w[i] = sol[i] #stores country i's equillibrium wage in the wage array
    welfare[i] = (sol[i] * L[i]) / priceIndex(i) #calculating and storing equilibrium welfare for country i (total income / price level)
    perCapitaWelfare[i] = sol[i] / priceIndex(i) #calculating and storing the equilibrium pre-capita welfare for country i
    print("country ")
    print(i)
    print("'s wage: ")
    println(w[i])
    print("welfare: ")
    println(welfare[i])
    print("welfare per capita: ")
    println(perCapitaWelfare[i])
    return nothing
end
    

#continuum of goods:
function f!(F, w) #the system of equations to optimize
    F[1] = (((T[1] * (1 * τ[1, 1])^-θ) / ϕ(1)) * 1 * L[1]) - (((T[1] * (1 * τ[1, 1])^-θ) / ϕ(1)) * 1 * L[1]) + sum((((T[1] * (1 * τ[i, 1])^-θ) / ϕ(i)) * w[i] * L[i]) - (((T[i] * (w[i] * τ[1, i])^-θ) / ϕ(1)) * 1 * L[1]) for i in 2:N)
    for n = 2:N
        F[n] = (((T[n] * (w[n] * τ[1, n])^-θ) / ϕ(1)) * 1 * L[1]) - (((T[1] * (1 * τ[n, 1])^-θ) / ϕ(n)) * w[n] * L[n]) + sum((((T[n] * (w[n] * τ[i, n])^-θ) / ϕ(i)) * w[i] * L[i]) - (((T[i] * (w[i] * τ[n, i])^-θ) / ϕ(n)) * w[n] * L[n]) for i in 2:N)
    end
end
results = nlsolve(f!, [1.0; 1.1; 1.2]) #solving for wages with country 1 set at 1.0
for i = 1:N
    sol[i] = results.zero[i] #saving results in sol[i]
    #println(sol[i])
end
for j = 1:N
    wageWelfare(j)
end

for n = 1:N
    for i = 1:N
        printTradeShares(n, i)
    end
end

display(w)

3-element Vector{Float64}:
 1.0
 0.7247796636776951
 0.7247796636776955

country 1's wage: 1.0
welfare: 1.9932340865428473
welfare per capita: 1.9932340865428473
country 2's wage: 0.7247796636776951
welfare: 1.5643270354951915
welfare per capita: 1.5643270354951915
country 3's wage: 0.7247796636776955
welfare: 1.6615150760853883
welfare per capita: 1.6615150760853883
fraction of goods country 1 buys from country 1:
0.40823669336460844
fraction of goods country 1 buys from country 2:
0.29588165331769617
fraction of goods country 1 buys from country 3:
0.29588165331769545
fraction of goods country 2 buys from country 1:
0.40823669336460844
fraction of goods country 2 buys from country 2:
0.29588165331769617
fraction of goods country 2 buys from country 3:
0.29588165331769545
fraction of goods country 3 buys from country 1:
0.40823669336460844
fraction of goods country 3 buys from country 2:
0.29588165331769617
fraction of goods country 3 buys from country 3:
0.29588165331769545
